## Coursera Capstone Project

### Scraping the New Orleans Wikipedia webpage and creating the dataset

In [81]:
import pandas as pd

In [82]:
import numpy as np

In [83]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


In [84]:
df=pd.read_html("https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans")[0]

In [85]:
df.head()

,Neighborhood,Longitude,Latitude
0,U.S. NAVAL BASE,-90.026093,29.946085
1,ALGIERS POINT,-90.051606,29.952462
2,WHITNEY,-90.042357,29.947200
3,AUDUBON,-90.121450,29.932994
4,OLD AURORA,-90.000000,29.924440


In [86]:
df.shape

(72, 3)

In [87]:
df_sorted = df.sort_values(by=['Neighborhood'])

In [88]:
df_sorted.head()

,Neighborhood,Longitude,Latitude
1,ALGIERS POINT,-90.051606,29.952462
3,AUDUBON,-90.121450,29.932994
5,B. W. COOPER,-90.091753,29.951774
6,BAYOU ST. JOHN,-90.086517,29.976071
7,BEHRMAN,-90.026436,29.934817


In [89]:
df_sorted.reset_index(drop=True, inplace=True)
df_sorted.shape

(72, 3)

### Segmenting and clustering New Orleans neighborhoods

In [90]:
import json # library to handle JSON files

!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium==0.5
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [91]:
address = 'New Orleans, LA'

geolocator = Nominatim(user_agent="nola_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Orleans, Louisiana are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Orleans, Louisiana are 29.9499323, -90.0701156.


In [92]:
# create map of New Orleans using latitude and longitude values
map_newOrleans = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_sorted['Latitude'], df_sorted['Longitude'], df_sorted['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newOrleans)  
    
map_newOrleans

In [93]:
print('The dataframe has {} neighborhoods.'.format(len(df_sorted['Neighborhood']),
        df_sorted.shape[0]
    )
)

The dataframe has 72 neighborhoods.


### Requesting information from Foursquare API

In [94]:
CLIENT_ID = 'PLVCWWLH4OLCZWPQ0XSVAOIHIBUF3ARXXT530SUHAJFKO5OJ' # your Foursquare ID
CLIENT_SECRET = '4CFUBKQ5LBVMZPZLHX1AVKAP2ANWGKO3POYMPZNARTAEUENC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: PLVCWWLH4OLCZWPQ0XSVAOIHIBUF3ARXXT530SUHAJFKO5OJ
CLIENT_SECRET:4CFUBKQ5LBVMZPZLHX1AVKAP2ANWGKO3POYMPZNARTAEUENC


In [95]:
df_sorted.loc[0, 'Neighborhood']

'ALGIERS POINT'

In [96]:
neighborhood_latitude = df_sorted.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_sorted.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_sorted.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ALGIERS POINT are 29.95246187, -90.0516057.


In [97]:
#GET request from Foursquare API for URL
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PLVCWWLH4OLCZWPQ0XSVAOIHIBUF3ARXXT530SUHAJFKO5OJ&client_secret=4CFUBKQ5LBVMZPZLHX1AVKAP2ANWGKO3POYMPZNARTAEUENC&ll=29.95246187,-90.0516057&v=20180605&radius=500&limit=100'

In [98]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600c8e6bef6ec337f1a96b00'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Algiers Point',
  'headerFullLocation': 'Algiers Point, New Orleans',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 29.956961874500006,
    'lng': -90.04642172567442},
   'sw': {'lat': 29.947961865499995, 'lng': -90.05678967432557}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba101aaf964a5204a8e37e3',
       'name': 'Tout de Suite Café',
       'location': {'address': '347 Verret St',
        'crossStreet': 'Alix',
        'lat': 29.95212113610269,
        'lng': -90.

In [99]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [100]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tout de Suite Café,Café,29.952121,-90.051090
1,Congregation Coffee Roasters,Coffee Shop,29.951918,-90.053395
2,The Crown & Anchor,Bar,29.951416,-90.054220
3,Old Point Bar,Bar,29.954940,-90.050226
4,Levee @ Algiers Point,Scenic Lookout,29.951760,-90.048747


In [101]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


### Exploring the neighborhoods in New Orleans

In [102]:
#Exploring neighbourhoods in New Orleans
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [103]:
newOrleans_venues = getNearbyVenues(names=df_sorted['Neighborhood'],
                                  latitudes=df_sorted['Latitude'],
                                  longitudes=df_sorted['Longitude']
                                  )

ALGIERS POINT
AUDUBON
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
BYWATER
CENTRAL BUSINESS DISTRICT
CENTRAL CITY
CITY PARK
DESIRE AREA
DILLARD
DIXON
EAST CARROLLTON
EAST RIVERSIDE
FAIRGROUNDS
FILMORE
FISCHER DEV
FLORIDA AREA
FLORIDA DEV
FRENCH QUARTER
FRERET
GARDEN DISTRICT
GENTILLY TERRACE
GENTILLY WOODS
GERT TOWN
HOLLYGROVE
HOLY CROSS
IBERVILLE
IRISH CHANNEL
LAKE CATHERINE
LAKE TERRACE & OAKS
LAKESHORE - LAKE VISTA
LAKEVIEW
LAKEWOOD
LEONIDAS
LITTLE WOODS
LOWER GARDEN DISTRICT
LOWER NINTH WARD
MARIGNY
MARLYVILLE - FONTAINEBLEAU
MID-CITY
MILAN
MILNEBURG
McDONOGH
NAVARRE
NEW AURORA - ENGLISH TURN
OLD AURORA
PINES VILLAGE
PLUM ORCHARD
PONTCHARTRAIN PARK
READ BLVD EAST
READ BLVD WEST
SEVENTH WARD
ST. ANTHONY
ST. BERNARD AREA
ST. CLAUDE
ST. ROCH
ST. THOMAS DEV
TALL TIMBERS - BRECHTEL
TOURO
TREME - LAFITTE
TULANE - GRAVIER
U.S. NAVAL BASE
UPTOWN
VIAVANT - VENETIAN ISLES
VILLAGE DE LEST
WEST END
WEST LAKE FOREST
WEST RIVERSIDE
WHITNEY


In [104]:
print(newOrleans_venues.shape)
newOrleans_venues.head()

(899, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALGIERS POINT,29.952462,-90.051606,Tout de Suite Café,29.952121,-90.051090,Café
1,ALGIERS POINT,29.952462,-90.051606,Congregation Coffee Roasters,29.951918,-90.053395,Coffee Shop
2,ALGIERS POINT,29.952462,-90.051606,The Crown & Anchor,29.951416,-90.054220,Bar
3,ALGIERS POINT,29.952462,-90.051606,Old Point Bar,29.954940,-90.050226,Bar
4,ALGIERS POINT,29.952462,-90.051606,Levee @ Algiers Point,29.951760,-90.048747,Scenic Lookout


In [105]:
newOrleans_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALGIERS POINT,20,20,20,20,20,20
AUDUBON,10,10,10,10,10,10
B. W. COOPER,5,5,5,5,5,5
BAYOU ST. JOHN,9,9,9,9,9,9
BLACK PEARL,5,5,5,5,5,5
...,...,...,...,...,...,...
UPTOWN,25,25,25,25,25,25
WEST END,2,2,2,2,2,2
WEST LAKE FOREST,11,11,11,11,11,11


In [31]:
print('There are {} unique categories.'.format(len(newOrleans_venues['Venue Category'].unique())))

There are 200 unique categories.


### Analyzing each neighborhood

In [106]:
#Analyzing each neighbourhood
# one hot encoding
newOrleans_onehot = pd.get_dummies(newOrleans_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newOrleans_onehot['Neighborhood'] = newOrleans_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newOrleans_onehot.columns[-1]] + list(newOrleans_onehot.columns[:-1])
newOrleans_onehot = newOrleans_onehot[fixed_columns]

newOrleans_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,...,Vegetarian / Vegan Restaurant,Vehicle Inspection Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
newOrleans_onehot.shape

(899, 200)

In [108]:
#calculating the mean of frequency of occurrence of each venue
newOrleans_grouped = newOrleans_onehot.groupby('Neighborhood').mean().reset_index()
newOrleans_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Vehicle Inspection Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,ALGIERS POINT,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0
1,AUDUBON,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
2,B. W. COOPER,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
3,BAYOU ST. JOHN,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
4,BLACK PEARL,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,UPTOWN,0.040000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.00,0.0
62,WEST END,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
63,WEST LAKE FOREST,0.000000,0.0,0.090909,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
64,WEST RIVERSIDE,0.035714,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0


In [109]:
newOrleans_grouped.shape

(66, 200)

In [110]:
#printing out top 5 venues of each neighborhood
num_top_venues = 5

for hood in newOrleans_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = newOrleans_grouped[newOrleans_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALGIERS POINT----
                     venue  freq
0                      Bar  0.15
1                     Park  0.10
2            Boat or Ferry  0.10
3  New American Restaurant  0.05
4           Scenic Lookout  0.05


----AUDUBON----
                   venue  freq
0            Coffee Shop   0.2
1  Outdoors & Recreation   0.1
2           Concert Hall   0.1
3                   Park   0.1
4          Smoothie Shop   0.1


----B. W. COOPER----
                  venue  freq
0          Home Service   0.2
1     Recreation Center   0.2
2  Gym / Fitness Center   0.2
3            Food Truck   0.2
4            Boxing Gym   0.2


----BAYOU ST. JOHN----
                    venue  freq
0       Convenience Store  0.11
1          Sandwich Place  0.11
2             Gas Station  0.11
3  Furniture / Home Store  0.11
4                    Park  0.11


----BLACK PEARL----
            venue  freq
0   Grocery Store   0.2
1           Plaza   0.2
2  Scenic Lookout   0.2
3  Farmers Market   0.2
4      Food Tr

In [111]:
#making a pandas dataframe out of thie venues / neighbourhood information
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newOrleans_grouped['Neighborhood']

for ind in np.arange(newOrleans_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newOrleans_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Bar,Boat or Ferry,Park,Pizza Place,New American Restaurant,Café,Food & Drink Shop,Scenic Lookout,Coffee Shop,Historic Site
1,AUDUBON,Coffee Shop,Concert Hall,Plaza,Smoothie Shop,Outdoors & Recreation,Park,Scenic Lookout,Sandwich Place,College Arts Building,Dessert Shop
2,B. W. COOPER,Recreation Center,Boxing Gym,Food Truck,Gym / Fitness Center,Home Service,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court
3,BAYOU ST. JOHN,Park,Convenience Store,Gas Station,Food Truck,Playground,Other Great Outdoors,Sandwich Place,Lounge,Furniture / Home Store,Women's Store
4,BLACK PEARL,Farmers Market,Food Truck,Grocery Store,Plaza,Scenic Lookout,Women's Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop


### Clustering the neighborhoods

In [113]:
#clustering the neighbourhoods
# set number of clusters
kclusters = 7

newOrleans_grouped_clustering = newOrleans_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newOrleans_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 3, 3], dtype=int32)

In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_
newOrleans_merged = df_sorted

# merge newOrleans_grouped with df_sorted to add latitude/longitude for each neighborhood
newOrleans_merged = newOrleans_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

newOrleans_merged.head() # check the last columns!

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,-90.051606,29.952462,2.0,Bar,Boat or Ferry,Park,Pizza Place,New American Restaurant,Café,Food & Drink Shop,Scenic Lookout,Coffee Shop,Historic Site
1,AUDUBON,-90.121450,29.932994,2.0,Coffee Shop,Concert Hall,Plaza,Smoothie Shop,Outdoors & Recreation,Park,Scenic Lookout,Sandwich Place,College Arts Building,Dessert Shop
2,B. W. COOPER,-90.091753,29.951774,2.0,Recreation Center,Boxing Gym,Food Truck,Gym / Fitness Center,Home Service,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court
3,BAYOU ST. JOHN,-90.086517,29.976071,2.0,Park,Convenience Store,Gas Station,Food Truck,Playground,Other Great Outdoors,Sandwich Place,Lounge,Furniture / Home Store,Women's Store
4,BEHRMAN,-90.026436,29.934817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
newOrleans_merged.shape

(72, 14)

In [116]:
newOrleans_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

In [117]:
newOrleans_merged.reset_index(drop=True, inplace=True)

In [118]:
newOrleans_merged.shape

(66, 14)

In [119]:
newOrleans_merged.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,-90.051606,29.952462,2.0,Bar,Boat or Ferry,Park,Pizza Place,New American Restaurant,Café,Food & Drink Shop,Scenic Lookout,Coffee Shop,Historic Site
1,AUDUBON,-90.121450,29.932994,2.0,Coffee Shop,Concert Hall,Plaza,Smoothie Shop,Outdoors & Recreation,Park,Scenic Lookout,Sandwich Place,College Arts Building,Dessert Shop
2,B. W. COOPER,-90.091753,29.951774,2.0,Recreation Center,Boxing Gym,Food Truck,Gym / Fitness Center,Home Service,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court
3,BAYOU ST. JOHN,-90.086517,29.976071,2.0,Park,Convenience Store,Gas Station,Food Truck,Playground,Other Great Outdoors,Sandwich Place,Lounge,Furniture / Home Store,Women's Store
4,BLACK PEARL,-90.134883,29.935895,2.0,Farmers Market,Food Truck,Grocery Store,Plaza,Scenic Lookout,Women's Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop


In [120]:
newOrleans_merged['Cluster Labels'] = newOrleans_merged['Cluster Labels'].astype(int)

In [121]:
newOrleans_merged.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,-90.051606,29.952462,2,Bar,Boat or Ferry,Park,Pizza Place,New American Restaurant,Café,Food & Drink Shop,Scenic Lookout,Coffee Shop,Historic Site
1,AUDUBON,-90.121450,29.932994,2,Coffee Shop,Concert Hall,Plaza,Smoothie Shop,Outdoors & Recreation,Park,Scenic Lookout,Sandwich Place,College Arts Building,Dessert Shop
2,B. W. COOPER,-90.091753,29.951774,2,Recreation Center,Boxing Gym,Food Truck,Gym / Fitness Center,Home Service,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court
3,BAYOU ST. JOHN,-90.086517,29.976071,2,Park,Convenience Store,Gas Station,Food Truck,Playground,Other Great Outdoors,Sandwich Place,Lounge,Furniture / Home Store,Women's Store
4,BLACK PEARL,-90.134883,29.935895,2,Farmers Market,Food Truck,Grocery Store,Plaza,Scenic Lookout,Women's Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop


In [122]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newOrleans_merged['Latitude'], newOrleans_merged['Longitude'], newOrleans_merged['Neighborhood'], newOrleans_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the individual clusters

In [123]:
#Examining the individual clusters
#Cluster 0 -- Rural - delta/swamp area of New Orleans. Clothing, women's store, shopping most common. Not much residential perhaps. More visitors. Food trucks most common eating. Could use another restaurant - perhaps organic/local to match rural.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 0, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,LAKE CATHERINE,Clothing Store,Women's Store,Farmers Market,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [124]:
#Examining the individual clusters
#Cluster 1 -- Outskirts scattered on fringe of midtown. Rural nature as well. More upscale because art gallery, trails. Suburb. Another French restaurant and fried chicken. Perhaps some other ethnic restaurant, maybe with kids menu, since playground and Sunday visitors. More well-to-do residential.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 1, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,LITTLE WOODS,Pharmacy,Seafood Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
42,McDONOGH,Dance Studio,Art Gallery,River,Playground,Seafood Restaurant,Women's Store,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck
52,ST. ANTHONY,Playground,Trail,Women's Store,Farmers Market,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [125]:
#Examining the individual clusters
#Cluster 2 -- Big midtown cluster with out-of-town tourists. Many restaurants, scenic places, bars, music venues. Mix of residential and tourist. Could use a delivery company here to handle overflow of eating demands.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 2, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Bar,Boat or Ferry,Park,Pizza Place,New American Restaurant,Café,Food & Drink Shop,Scenic Lookout,Coffee Shop,Historic Site
1,AUDUBON,Coffee Shop,Concert Hall,Plaza,Smoothie Shop,Outdoors & Recreation,Park,Scenic Lookout,Sandwich Place,College Arts Building,Dessert Shop
2,B. W. COOPER,Recreation Center,Boxing Gym,Food Truck,Gym / Fitness Center,Home Service,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court
3,BAYOU ST. JOHN,Park,Convenience Store,Gas Station,Food Truck,Playground,Other Great Outdoors,Sandwich Place,Lounge,Furniture / Home Store,Women's Store
4,BLACK PEARL,Farmers Market,Food Truck,Grocery Store,Plaza,Scenic Lookout,Women's Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop
5,BROADMOOR,Music Venue,Taco Place,Coffee Shop,Food Truck,Recreation Center,Hotel Bar,Ethiopian Restaurant,French Restaurant,Ice Cream Shop,Food Court
6,BYWATER,Music Venue,Discount Store,Caribbean Restaurant,Seafood Restaurant,Coffee Shop,Park,Vegetarian / Vegan Restaurant,Hotel,Indie Theater,Antique Shop
7,CENTRAL BUSINESS DISTRICT,Hotel,Hotel Bar,Bar,Café,Coffee Shop,Restaurant,Cocktail Bar,Juice Bar,Seafood Restaurant,Shipping Store
11,DILLARD,Women's Store,Discount Store,Donut Shop,Mobile Phone Shop,Tourist Information Center,Gym / Fitness Center,Flower Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant
12,DIXON,Thrift / Vintage Store,Smoothie Shop,Warehouse Store,Food Court,Optical Shop,Food,Pharmacy,Farmers Market,Fried Chicken Joint,French Restaurant


In [126]:
#Examining the individual clusters
#Cluster 3 -- Part cluster in center midtown, and part out at the big lake. Recreation area, tourists plus local Sunday visitors. Could use a different ethnic restaurant to complement what exists. Would be plenty of demand I think.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 3, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,CENTRAL CITY,Park,Cajun / Creole Restaurant,Fried Chicken Joint,Grocery Store,Seafood Restaurant,Garden,French Restaurant,Food Truck,Food Court,Food & Drink Shop
9,CITY PARK,Baseball Field,Trail,Tennis Court,Park,Women's Store,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck
31,LAKESHORE - LAKE VISTA,Park,Food,Harbor / Marina,Women's Store,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
40,MILAN,Wine Shop,Theater,Dive Bar,Park,Women's Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
48,PONTCHARTRAIN PARK,Park,Women's Store,Fast Food Restaurant,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
54,ST. CLAUDE,Plaza,Park,Women's Store,Farmers Market,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [127]:
#Examining the individual clusters
#Cluster 4 -- Lone Desire area on outskirts of New Orleans. Perhaps lower class residents, because lots of fast food, food trucks. Maybe needs a couple of affordable family restaurants.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 4, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,DESIRE AREA,Skate Park,Women's Store,Fast Food Restaurant,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [128]:
#Examining the individual clusters
#Cluster 5 -- Lone neighborhood on east side of New Orleans, near the highway to out of town. Probably residential, lower middle class. Gas station main venue -- for transient traffic. Could use a family restaurant.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 5, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,READ BLVD WEST,Gas Station,Women's Store,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food


In [129]:
#Examining the individual clusters
#Cluster 6 -- Between town and lake. Historic site, event space for special concerts or such. Must experience many out-of-towners. French restaurant is the only one. Could use another local - maybe cajun restaurant.
newOrleans_merged.loc[newOrleans_merged['Cluster Labels'] == 6, newOrleans_merged.columns[[0] + list(range(4, newOrleans_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,WEST END,Historic Site,Playground,Event Space,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food


### Listing neighborhoods not found by Foursquare API

In [143]:
import pandas as pd
#missing neighborhoods not returned by Foursquare API
data = {'Neighborhood': ['Behrman','Village de Lest','Viavant - Venetian Isles','Tall Timbers - Brechtel','Florida Dev','Lower Ninth Ward'],
           'Longitude': [ -90.02643585, -89.90112305,-89.945755,-90.01338959,-90.03252983,-90.01304626],
           'Latitude': [ 29.9348167, 30.07325354, 30.00311157, 29.91030537,29.98170245, 29.96891424]
             }
df_missing = pd.DataFrame(data)

In [144]:
df_missing.head(6)

,Neighborhood,Longitude,Latitude
0,Behrman,-90.026436,29.934817
1,Village de Lest,-89.901123,30.073254
2,Viavant - Venetian Isles,-89.945755,30.003112
3,Tall Timbers - Brechtel,-90.013390,29.910305
4,Florida Dev,-90.032530,29.981702
5,Lower Ninth Ward,-90.013046,29.968914


In [145]:
address = 'New Orleans, LA'

geolocator = Nominatim(user_agent="nola_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Orleans, Louisiana are {}, {}.'.format(latitude, longitude))

# create map of missing neighborhoods not returned by Foursquare API New Orleans using latitude and longitude values
map_newOrleans_missing = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_missing['Latitude'], df_missing['Longitude'], df_missing['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newOrleans_missing)  
    
map_newOrleans_missing


The geograpical coordinate of New Orleans, Louisiana are 29.9499323, -90.0701156.
